<a href="https://colab.research.google.com/github/elemnurguner/data-ai-projects/blob/main/Dondurulmu%C5%9FG%C4%B1dalar%C4%B1nSonKullanmaTahmini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np

# Rastgele veri oluşturmak için seed belirleme
np.random.seed(42)

# Veri seti boyutu
num_samples = 1000

# Gıda türleri
food_types = ['Et', 'Sebze', 'Meyve', 'Balık', 'Tavuk']

# Paketleme türleri
packaging_types = ['Vakum', 'Plastik', 'Kağıt', 'Karton']

# Rastgele veri oluşturma
data = {
    'Gıda_Türü': np.random.choice(food_types, num_samples),
    'Paketleme_Türü': np.random.choice(packaging_types, num_samples),
    'Sıcaklık': np.random.uniform(-25, -15, num_samples),  # Dondurucu sıcaklık
    'Nem': np.random.uniform(70, 95, num_samples),        # Nem oranı
    'pH': np.random.uniform(4.0, 7.0, num_samples),       # pH değeri
    'Mikrobiyal_Yük': np.random.randint(100, 10000, num_samples),  # Mikrobiyal yük
}

# Raf ömrü hesaplama (örnek bir formül)
# Bu formül, sıcaklık, nem, pH ve mikrobiyal yüke bağlı olarak raf ömrünü tahmin eder.
data['Raf_Ömrü'] = (
    365  # Temel raf ömrü (gün)
    - (data['Sıcaklık'] + 20) * 10  # Sıcaklık etkisi
    - (data['Nem'] - 80) * 2        # Nem etkisi
    - (data['pH'] - 6.0) * 30       # pH etkisi
    - data['Mikrobiyal_Yük'] / 100  # Mikrobiyal yük etkisi
)

# Veriyi DataFrame'e dönüştürme
df = pd.DataFrame(data)

# Raf ömrünü pozitif yapma (negatif değerleri düzeltme)
df['Raf_Ömrü'] = df['Raf_Ömrü'].apply(lambda x: max(x, 30))  # Minimum 30 gün

# Veri setini göster
print(df.head())

# Veri setini CSV olarak kaydetme
df.to_csv('dondurulmus_gida_veri_seti.csv', index=False)


  Gıda_Türü Paketleme_Türü   Sıcaklık        Nem        pH  Mikrobiyal_Yük  \
0     Balık         Karton -23.325174  75.476720  5.944954            8850   
1     Tavuk         Karton -23.954322  70.918034  4.222316            7454   
2     Meyve         Karton -18.635698  72.700644  5.126407            4616   
3     Tavuk         Karton -17.935243  78.471516  6.411444            3722   
4     Tavuk         Karton -24.684139  90.064642  5.300417            4499   

     Raf_Ömrü  
0  320.449681  
1  401.497660  
2  346.003484  
3  297.846086  
4  367.709601  


burda kendi veri setimizi kendimiz oluşturduk .

In [9]:
import pandas as pd

# Veri setini yükleme
df = pd.read_csv('dondurulmus_gida_veri_seti.csv')

# İlk 5 satırı göster
print(df.head())

  Gıda_Türü Paketleme_Türü   Sıcaklık        Nem        pH  Mikrobiyal_Yük  \
0     Balık         Karton -23.325174  75.476720  5.944954            8850   
1     Tavuk         Karton -23.954322  70.918034  4.222316            7454   
2     Meyve         Karton -18.635698  72.700644  5.126407            4616   
3     Tavuk         Karton -17.935243  78.471516  6.411444            3722   
4     Tavuk         Karton -24.684139  90.064642  5.300417            4499   

     Raf_Ömrü  
0  320.449681  
1  401.497660  
2  346.003484  
3  297.846086  
4  367.709601  


2. Veri Ön İşleme
Veri setindeki kategorik değişkenleri (örneğin, Gıda_Türü ve Paketleme_Türü) sayısallaştıralım ve sayısal değişkenleri normalize edelim.

In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Özellikler ve hedef değişken
X = df.drop('Raf_Ömrü', axis=1)  # Özellikler
y = df['Raf_Ömrü']               # Hedef değişken

# Kategorik ve sayısal özellikleri ayırma
categorical_features = ['Gıda_Türü', 'Paketleme_Türü']
numerical_features = ['Sıcaklık', 'Nem', 'pH', 'Mikrobiyal_Yük']

# Ön işleme adımları
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),  # Kategorik özellikleri one-hot encoding
        ('num', StandardScaler(), numerical_features)    # Sayısal özellikleri standartlaştırma
    ])

# Veriyi ön işleme
X_processed = preprocessor.fit_transform(X)

# İşlenmiş veriyi kontrol etme
print(X_processed[:5])

[[ 1.          0.          0.          0.          0.          1.
   0.          0.          0.         -1.16638273 -0.95377374  0.54310617
   1.34414585]
 [ 0.          0.          0.          0.          1.          1.
   0.          0.          0.         -1.38467554 -1.58522499 -1.46783208
   0.85538247]
 [ 0.          0.          1.          0.          0.          1.
   0.          0.          0.          0.46070656 -1.3383049  -0.41243306
  -0.13824966]
 [ 0.          0.          0.          0.          1.          1.
   0.          0.          0.          0.70374059 -0.53894639  1.08766744
  -0.45125429]
 [ 0.          0.          0.          0.          1.          1.
   0.          0.          0.         -1.63789726  1.06688774 -0.20930086
  -0.17921335]]


3. Veriyi Eğitim ve Test Setlerine Ayırma
Veri setini eğitim ve test setlerine ayıralım.

In [11]:
from sklearn.model_selection import train_test_split

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")

Eğitim seti boyutu: (800, 13)
Test seti boyutu: (200, 13)


4. Model Eğitimi
Bu örnekte, bir Random Forest Regresyon modeli kullanacağız.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Model oluşturma
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Modeli eğitme
model.fit(X_train, y_train)

# Eğitim seti üzerinde tahmin yapma
y_train_pred = model.predict(X_train)

# Test seti üzerinde tahmin yapma
y_test_pred = model.predict(X_test)

5. Model Performansını Değerlendirme
Modelin performansını değerlendirmek için Ortalama Kare Hatası (MSE) ve R² Skoru kullanacağız.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Eğitim seti performansı
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Test seti performansı
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Eğitim Seti MSE: {train_mse}")
print(f"Eğitim Seti R²: {train_r2}")
print(f"Test Seti MSE: {test_mse}")
print(f"Test Seti R²: {test_r2}")

Eğitim Seti MSE: 14.627210763642074
Eğitim Seti R²: 0.9940854373183682
Test Seti MSE: 97.4374878212823
Test Seti R²: 0.9680912952437766


6. Tahmin Yapma
Eğitilen modeli kullanarak yeni veriler üzerinde tahmin yapalım.

In [ ]:
# Yeni bir örnek veri
new_data = pd.DataFrame({
    'Gıda_Türü': ['Et'],
    'Paketleme_Türü': ['Vakum'],
    'Sıcaklık': [-18],
    'Nem': [85],
    'pH': [6.5],
    'Mikrobiyal_Yük': [1000]
})

# Veriyi ön işleme
new_data_processed = preprocessor.transform(new_data)

# Tahmin yapma
predicted_shelf_life = model.predict(new_data_processed)
print(f"Tahmini Raf Ömrü: {predicted_shelf_life[0]:.2f} gün")

Tahmini Raf Ömrü: 311.24 gün


7. Modeli Geliştirme
Modelin performansını artırmak için şu adımları deneyebilirsiniz:

Hiperparametre Optimizasyonu: GridSearchCV veya RandomizedSearchCV kullanarak en iyi hiperparametreleri bulun.

Farklı Modeller Deneyin: Gradient Boosting, XGBoost veya derin öğrenme modelleri kullanın.

Daha Fazla Veri Toplayın: Veri setinizi genişleterek modelin doğruluğunu artırın.

1. Hiperparametre Optimizasyonu
Hiperparametre optimizasyonu, modelin en iyi performansı göstermesi için en uygun hiperparametreleri bulma işlemidir. Bunun için GridSearchCV veya RandomizedSearchCV kullanabilirsiniz.

GridSearchCV ile Hiperparametre Optimizasyonu

In [ ]:
from sklearn.model_selection import GridSearchCV

# Hiperparametre grid'i
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV ile model oluşturma
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,  # 5 katlı çapraz doğrulama
    scoring='neg_mean_squared_error',  # MSE'yi minimize et
    n_jobs=-1  # Tüm CPU çekirdeklerini kullan
)

# GridSearchCV'yi eğitme
grid_search.fit(X_train, y_train)

# En iyi hiperparametreler
print("En iyi hiperparametreler:", grid_search.best_params_)

# En iyi model
best_model = grid_search.best_estimator_

# Test seti üzerinde tahmin yapma
y_test_pred = best_model.predict(X_test)

# Performans değerlendirme
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test Seti MSE: {test_mse}")
print(f"Test Seti R²: {test_r2}")

En iyi hiperparametreler: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Test Seti MSE: 96.80903689115166
Test Seti R²: 0.9682970995561794


RandomizedSearchCV ile Hiperparametre Optimizasyonu

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Hiperparametre dağılımları
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 4)
}

# RandomizedSearchCV ile model oluşturma
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=20,  # Denenecek kombinasyon sayısı
    cv=5,  # 5 katlı çapraz doğrulama
    scoring='neg_mean_squared_error',  # MSE'yi minimize et
    n_jobs=-1  # Tüm CPU çekirdeklerini kullan
)

# RandomizedSearchCV'yi eğitme
random_search.fit(X_train, y_train)

# En iyi hiperparametreler
print("En iyi hiperparametreler:", random_search.best_params_)

# En iyi model
best_model = random_search.best_estimator_

# Test seti üzerinde tahmin yapma
y_test_pred = best_model.predict(X_test)

# Performans değerlendirme
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test Seti MSE: {test_mse}")
print(f"Test Seti R²: {test_r2}")

En iyi hiperparametreler: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 185}
Test Seti MSE: 98.68444402767363
Test Seti R²: 0.9676829436090684


2. Farklı Modeller Deneyin
Random Forest dışında farklı modeller de deneyebilirsiniz. Örneğin, Gradient Boosting veya XGBoost gibi gelişmiş modeller kullanabilirsiniz.

Gradient Boosting Regresyon

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting modeli
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Modeli eğitme
gb_model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
y_test_pred = gb_model.predict(X_test)

# Performans değerlendirme
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test Seti MSE: {test_mse}")
print(f"Test Seti R²: {test_r2}")

Test Seti MSE: 41.653042115273884
Test Seti R²: 0.9863595146716774


XGBoost Regresyon

In [ ]:
from xgboost import XGBRegressor

# XGBoost modeli
xgb_model = XGBRegressor(n_estimators=100, random_state=42)

# Modeli eğitme
xgb_model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
y_test_pred = xgb_model.predict(X_test)

# Performans değerlendirme
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test Seti MSE: {test_mse}")
print(f"Test Seti R²: {test_r2}")

Test Seti MSE: 89.73732868004227
Test Seti R²: 0.970612933579367


4. Derin Öğrenme Modelleri Deneyin
Daha karmaşık modeller için derin öğrenme kullanabilirsiniz. Örneğin, bir sinir ağı (neural network) modeli eğitebilirsiniz.

Basit Bir Sinir Ağı Modeli

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Sinir ağı modeli
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Giriş katmanı
model.add(Dense(32, activation='relu'))  # Gizli katman
model.add(Dense(1))  # Çıkış katmanı

# Modeli derleme
model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğitme
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Test seti üzerinde tahmin yapma
y_test_pred = model.predict(X_test).flatten()

# Performans değerlendirme
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test Seti MSE: {test_mse}")
print(f"Test Seti R²: {test_r2}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 107809.2344 - val_loss: 104936.1484
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 107640.0781 - val_loss: 103320.0234
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 105972.3359 - val_loss: 100432.7109
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 101860.9531 - val_loss: 95601.8359
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 97128.3125 - val_loss: 88141.7734
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 88320.1016 - val_loss: 77616.1172
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 76804.5391 - val_loss: 64073.5430
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 60659.3203 - val_loss: 48342.9883
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 46135.7109 - val_loss: 32164.7812
Epoch 10/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29369.4961 - val_loss: 18004.9219
Epoch 11/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15710.1377 - val_loss: 

5. Sonuçları Karşılaştırın
Farklı modellerin performanslarını karşılaştırarak en iyi modeli seçin. Örneğin:

Random Forest

Gradient Boosting

XGBoost

Sinir Ağı

Bu modellerin MSE ve R² değerlerini karşılaştırarak hangisinin daha iyi performans gösterdiğini belirleyebilirsiniz.

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import numpy as np

def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    explained_variance = explained_variance_score(y_true, y_pred)

    print(f"{model_name} Performansı:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R²: {r2:.2f}")
    print(f"Açıklanmış Varyans: {explained_variance:.2f}")
    print("-" * 30)

# --------------------------------------------
# DÜZELTME: TÜM TAHMİN DEĞİŞKENLERİNİ TANIMLA
# --------------------------------------------

# Random Forest Tahminleri (DÜZELTME: y_test_pred_rf tanımlandı)
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_test_pred_rf = rf_model.predict(X_test)  # Değişken adı düzeltildi ✅

# Gradient Boosting Tahminleri (DÜZELTME: y_test_pred_gb tanımlandı)
from sklearn.ensemble import GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_test_pred_gb = gb_model.predict(X_test)  # Değişken adı düzeltildi ✅

# XGBoost Tahminleri (DÜZELTME: y_test_pred_xgb tanımlandı)
from xgboost import XGBRegressor
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_test_pred_xgb = xgb_model.predict(X_test)  # Değişken adı düzeltildi ✅

# Sinir Ağı Tahminleri (DÜZELTME: y_test_pred_nn tanımlandı)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
nn_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1)
])
nn_model.compile(optimizer='adam', loss='mse')
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
y_test_pred_nn = nn_model.predict(X_test).flatten()  # Değişken adı düzeltildi ✅

# --------------------------------------------
# PERFORMANS DEĞERLENDİRME (ARTIK HATA YOK)
# --------------------------------------------
evaluate_model(y_test, y_test_pred_rf, "Random Forest")
evaluate_model(y_test, y_test_pred_gb, "Gradient Boosting")
evaluate_model(y_test, y_test_pred_xgb, "XGBoost")
evaluate_model(y_test, y_test_pred_nn, "Sinir Ağı")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Random Forest Performansı:
MSE: 97.44
RMSE: 9.87
MAE: 7.83
R²: 0.97
Açıklanmış Varyans: 0.97
------------------------------
Gradient Boosting Performansı:
MSE: 41.65
RMSE: 6.45
MAE: 5.14
R²: 0.99
Açıklanmış Varyans: 0.99
------------------------------
XGBoost Performansı:
MSE: 89.74
RMSE: 9.47
MAE: 7.68
R²: 0.97
Açıklanmış Varyans: 0.97
------------------------------
Sinir Ağı Performansı:
MSE: 22.90
RMSE: 4.79
MAE: 3.41
R²: 0.99
Açıklanmış Varyans: 0.99
------------------------------
